In [1]:
# !pip install gymnasium renderlab
# !pip install opencv-python
# !pip install pygame

In [2]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [14]:
#visualise maze:
rfpMaze = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
maze1 = ["SFFH", "FHHF", "FHFG", "FFFH", "HFHH"]

desc = maze1
mazeSize = [len(desc),len(desc[0])]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = -1
gift_found = False
for i in range(len(desc)):
    if gift_found:
        break
    for j in range(len(desc[i])):
        giftState += 1
        if desc[i][j] == 'G':
            gift_found = True
            break
            
print(giftState)
print(statePositions)

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

11
[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]


In [19]:
# Q-LEARNING FUNCTIONS

epsilonValue = 0.5
alpha = 0.5
qTable_1 = {}
currentState = 0

def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    if changeInDist > 0:
        return changeInDist/2
    else:
        return changeInDist

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 0:
        return [1,2]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]
        maxValue = max(qValues.values())
        count = sum(1 for value in qValues.values() if value == maxValue)
        if count > 1:
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def canConverge():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1])
        if max(qTable_1[currentState]) > 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0:
            newState = currentState - 1
        elif bestDirection == 1:
            newState = currentState + mazeSize[1]
        elif bestDirection == 2:
            newState = currentState + 1
        elif bestDirection == 3:
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return true
        currentState = newState
    return false

convergenceThresh = 0.00001
def updateTable(direction, nextState, reward):
    global qTable_1
    didConverge = False
    global currentState
    updated = qTable_1[currentState][direction] + alpha*(reward + max(qTable_1[nextState]) - qTable_1[currentState][direction])
    changeInQ = abs(qTable_1[currentState][direction] - updated)
    if changeInQ < convergenceThresh:
        if changeInQ > 0 and canConverge():
            didConverge = True
    qTable_1[currentState][direction] = updated
    currentState = nextState
    return didConverge, changeInQ

SyntaxError: expected ':' (2183508386.py, line 77)

In [20]:
maxEpisodes = 1000
currentEpisode = 1
converged = False

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    if converged:
        break
        
    direction = nextStep(currentState)
    nextState, reward, terminated, truncated, info = env.step(direction)
    
    if terminated:
        if reward < 1:
            reward = -10
            
    if not terminated:
        reward = calcReward(currentState, nextState)
    
    converged, changeInQ = updateTable(direction, nextState, reward)

    if terminated or truncated or converged:
        nextState, info = env.reset()
        if not converged:
            currentEpisode += 1

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable_1)):
        print(str(i) + ": " + str(qTable_1[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 15/1000
Time: 26.595 sec
Q-Table:
0: [0, 0.3360419451020363, 0.6833378337839389, 0]
1: [-0.11211204794381394, -9.859958076926594, 0.2961705349399232, 0]
2: [-0.29560109789346745, -9.375, -7.5, 0]
3: [-0.6327797297624761, -0.5811388300841898, 0, 0]
4: [0, 0.07860324164405882, -8.828672699243661, 0.19381049914450071]
5: [-0.5678771973185213, -0.5924846300375881, -0.26168795823788127, -0.5390308707225124]
6: [-1.1559896357093853, 0, -0.5590260330582245, -0.7603346045991202]
7: [0, 0, 0, 0]
8: [0, 0, 0, -0.08494221274438615]
9: [0, 0, 0, 0]
10: [0, 0, 0, 0]
11: [0, 0, 0, 0]
12: [0, 0, 0, 0]
13: [0, 0, 0, 0]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0, 0]
17: [0, 0, 0, 0]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 9.038683277062809e-06
26.594 seconds to converge


In [ ]:
env.reset()

In [ ]:
env.close()